In [1]:
%matplotlib inline
import numpy as np
from scipy.optimize import root_scalar
from math import log, ceil
import ipywidgets as widgets
import matplotlib.pyplot as plt

class StabilityPredictor:
    
    def __init__(self, I, E, N, P, R, g, a, h, rI, rE, rY):      
        self.I=I
        self.E=E
        self.N=N
        self.P=P
        self.R=R
        self.g=g
        self.a=a
        self.h=h
        self.rI=rI
        self.rE=rE
        self.rY=rY
    
    def run(self):
        if not self.check_ss():
            return np.inf, 'FAIL_SS'
        elif not self.check_initial_condition():
            return np.inf, 'FAIL_IC'
        current_domain = self.determine_domain();
        time_in_danger = 0
        # Placeholder values to begin while loop
        done = False 
        while not done:
            #Used to calculate new E and I values at crossover point
            temp_E = self.E
            temp_I = self.I
            if current_domain == 'mentee':
                t1 = self.find_mentee_yfr_crossover()
                t2 = self.find_mentee_mentor_crossover()
                t = min((i for i in (t1,t2) if i>0), default=-1)
                if t == -1:
                    done = True
                else:
                    self.E = self.E_mentee_lim(temp_E, temp_I, t)
                    self.I = self.I_mentee_lim(temp_E, temp_I, t)
                    if t == t1:
                        current_domain = 'yfr'
                    else:
                        current_domain = 'mentor'
                        if self.check_under_crit_value():
                            done = True
            elif current_domain == 'mentor':
                t1 = self.find_mentor_yfr_crossover()
                t2 = self.find_mentor_mentee_crossover()
                t = min((i for i in (t1,t2) if i > 0), default=-1)
                if t == -1:
                    done = True
                else:
                    self.E = self.E_mentor_lim(temp_E, temp_I, t)
                    self.I = self.I_mentor_lim(temp_E, temp_I, t)
                    time_in_danger += t
                    if t == t1:
                        current_domain = 'yfr'
                    else:
                        current_domain = 'mentee'
            else:
                t1 = self.find_yfr_mentee_crossover()
                t2 = self.find_yfr_mentor_crossover()
                t = min((i for i in (t1,t2) if i > 0), default=-1)
                if t==-1:
                    done = True
                else:
                    time_in_danger += t
                    self.E = self.E_yfr_lim(temp_E, t)
                    self.I = self.I_yfr_lim(temp_I, t)
                    if t == t1:
                        current_domain = 'mentee' 
                    else:
                        current_domain = 'mentor'
                        if self.check_under_crit_value():
                            done = True
        
        if current_domain == 'mentee':
            return time_in_danger, 'PASS'
        elif current_domain == 'yfr':
            return np.inf, 'FAIL_YFR'
        else:
            return np.inf, 'FAIL_MENTOR'
        
                
    def check_ss(self):
        try:
            Iss = self.g*self.R/self.rI
            Ess = (self.g-self.h-self.a*self.N)/self.a
            return Iss>0 and Ess>0 and Iss+Ess<self.P and (self.rE*Ess>self.rI*Iss or self.rE*Ess>self.rY) and\
                (self.rY>self.rI*Iss or self.rY>self.rE*Ess)
        except:
            return False

    def check_initial_condition(self):
        return self.rE*self.E>self.rI*self.I or self.rE*self.E>self.rY or self.E>((self.h+self.a*self.N)/(self.rE/self.R-self.a))
        
    def determine_domain(self):
        current_min = min(self.rI*self.I, self.rE*self.E, self.rY)
        if self.rI*self.I == current_min:
            return 'mentee'
        elif self.rE*self.E == current_min:
            return 'mentor'
        else:
            return 'yfr'
    

    def find_yfr_mentee_crossover(self):
        try:
            return (self.rY/self.rI-self.I)*(self.R/(self.g-self.rY))
        except:
            return -1

    def find_yfr_mentor_crossover(self):
        try:
            k=-self.rY+self.h*self.R+self.a*self.N*self.R
            log_arg = (self.rY*self.a*self.R+self.rE*self.k)/(self.rE*(self.E*self.a*self.R+k))
            return -log(log_arg)/self.a
        except:
            return -1

    def find_mentee_yfr_crossover(self):
        try:
            k1=self.I-self.g*self.R/self.rI
            log_arg = (self.rY-self.g*self.R)/(self.rI*self.k1)
            return -self.R*log(log_arg)/self.rI
        except:
            return -1

    def find_mentee_mentor_crossover(self):
        try:
            k1=self.I-self.g*self.R/self.rI
            k2=self.E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
            func = lambda t: self.rI*(k1*np.exp(-self.rI*t/self.R)+self.g*self.R/self.rI)-\
                self.rE*(-self.rI/(self.rI-self.a*self.R)*k1*np.exp(-self.rI*t/self.R)+k2*np.exp(-self.a*t)+\
                (self.g-self.h-self.a*self.N)/self.a)
            t_sol = root_scalar(func, bracket=[0.01, self.get_bracket_upper('mentee')])
            return t_sol.root
        except:
            return -1

    def find_mentor_yfr_crossover(self):
        k1=self.E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
        k2=self.I+k1/(1-self.a*self.R/self.rE)
        try:
            log_arg = (self.rY-(self.rE*(self.h+self.a*self.N))/(self.rE/self.R-self.a))/(self.rE*k1)
            return log(log_arg)/(self.rE/self.R-self.a)
        except:
            return -1

    def find_mentor_mentee_crossover(self):
        try:
            k1=self.E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
            k2=self.I+k1/(1-self.a*self.R/self.rE)
            func = lambda t: self.rE*(k1*np.exp((self.rE/self.R-self.a)*t)+(self.h+self.a*self.N)/(self.rE/self.R-self.a)) -\
                self.rI*(-k1/(1-self.a*self.R/self.rE)*np.exp((self.rE/self.R-self.a)*t)+\
                ((-self.h-self.a*self.N)/(1-self.a*self.R/self.rE)+self.g)*t+k2)
            t_sol = root_scalar(func, bracket=[0.01, self.get_bracket_upper('mentor')])
            return t_sol.root
        except:
            return -1
    
    def get_bracket_upper(self, domain):
        if domain == 'mentee':
            k1=self.I-self.g*self.R/self.rI
            k2=self.E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
            func = lambda t: self.rI*(k1*np.exp(-self.rI*t/self.R)+self.g*self.R/self.rI)-\
                self.rE*(-self.rI/(self.rI-self.a*self.R)*k1*np.exp(-self.rI*t/self.R)+k2*np.exp(-self.a*t)+\
                (self.g-self.h-self.a*self.N)/self.a)
        else:
            k1=self.E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
            k2=self.I+k1/(1-self.a*self.R/self.rE)
            func = lambda t: self.rE*(k1*np.exp((self.rE/self.R-self.a)*t)+(self.h+self.a*self.N)/(self.rE/self.R-self.a)) -\
                self.rI*(-k1/(1-self.a*self.R/self.rE)*np.exp((self.rE/self.R-self.a)*t)+\
                ((-self.h-self.a*self.N)/(1-self.a*self.R/self.rE)+self.g)*t+k2)
        return self.find_sign_change(func)
        
    def find_sign_change(self, func):
        i = 1
        done = False
        # Checking for sign change within next 40 years
        while i < 2080 and done == False:
            if func(i) > 0:
                done = True
            i+=1
        return i
    
    def check_under_crit_value(self):
        return self.E < (self.h+self.a*self.N)/(self.rE/self.R-self.a)
    
    def E_mentor_lim(self, E, I, t):
        k1=E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
        k2=I+k1/(1-self.a*self.R/self.rE)
        return k1*np.exp((self.rE/self.R-self.a)*t)+(self.h+self.a*self.N)/(self.rE/self.R-self.a)
    
    def I_mentor_lim(self, E, I, t):
        k1=E-(self.h+self.a*self.N)/(self.rE/self.R-self.a)
        k2=I+k1/(1-self.a*self.R/self.rE)
        return -k1/(1-self.a*self.R/self.rE)*np.exp((self.rE/self.R-self.a)*t)+\
                ((-self.h-self.a*self.N)/(1-self.a*self.R/self.rE)+self.g)*t+k2
    
    def E_mentee_lim(self, E, I, t):
        k1=I-self.g*self.R/self.rI
        k2=E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
        return -self.rI/(self.rI-self.a*self.R)*k1*np.exp(-self.rI*t/self.R)+k2*np.exp(-self.a*t)+\
                (self.g-self.h-self.a*self.N)/self.a
    
    def I_mentee_lim(self, E, I, t):
        k1=I-self.g*self.R/self.rI
        k2=E+(self.rI/(self.rI-self.a*self.R))*k1-(self.g-self.h-self.a*self.N)/self.a
        return k1*np.exp(-self.rI*t/self.R)+self.g*self.R/self.rI
    
    def E_yfr_lim(self, E, t):
        k=-rY+h*R+a*N*R
        return ((E*self.a*self.R+k)*np.exp(-self.a*t)-k)/(self.a*self.R)
    
    def I_yfr_lim(self, I, t):
        return I+(self.g-self.rY/self.R)*t
    

In [35]:
predictor = StabilityPredictor(I=1, E=20, N=25, P=30, R=250, g=4/52, a=0.07/52, h=1/52, rI=144/52, rE=144/52, rY=10000/52)
predictor.run()

(0, 'PASS')

In [2]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

def plot_predictions(N=25, P=30, R=250, g=4, a=0.07,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,5)

    xpass = []
    ypass = []
    xssfail = []
    yssfail = []
    xicfail = []
    yicfail = []
    xmentorfail = []
    ymentorfail = []
    xyfr = []
    yyfr = []
    for i in range(0, P+1):
        for e in range(0, P+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, status = predictor.run()
            if status == 'PASS':
                xpass.append(i)
                ypass.append(e)
            elif status == 'FAIL_SS':
                xssfail.append(i)
                yssfail.append(e)
            elif status == 'FAIL_IC':
                xicfail.append(i)
                yicfail.append(e)
            elif status == 'FAIL_MENTOR':
                xmentorfail.append(i)
                ymentorfail.append(e)
            else:
                xyfr.append(i)
                yyfr.append(e)


    plt.scatter(xpass, ypass, c='green', label='Successes')
    plt.scatter(xssfail, yssfail, c='red', label='SS Failures')
    plt.scatter(xicfail, yicfail, c='firebrick', label='IC Failures')
    plt.scatter(xmentorfail, ymentorfail, c='lightcoral', label='Mentor-Limited Crashes')
    plt.scatter(xyfr, yyfr, c='blue', label='YFR')
    xP = np.array(P)
    plt.axline((0,P), slope=-1, linestyle='dashed', label='E+I=P', c='black')
    plt.legend()

    plt.title('Predicted Stability of the System Depending on \nInitial Experienced and Inexperienced Pilots')
    plt.xlabel('I0')
    plt.ylabel('E0')
  
widgets.interact_manual(plot_predictions, N=widgets.IntSlider(value=25, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000))



interactive(children=(IntSlider(value=25, description='N'), IntSlider(value=30, description='P'), IntSlider(va…

<function __main__.plot_predictions(N=25, P=30, R=250, g=4, a=0.07, h=1, rI=144, rE=144, rY=10000)>

In [7]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

def plot_predictions(i=10, N=25, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,5)

    xpass = []
    ypass = []
    xssfail = []
    yssfail = []
    xicfail = []
    yicfail = []
    xmentorfail = []
    ymentorfail = []
    xyfr = []
    yyfr = []
    poss_a_values = [x/100 for x in range(0,30)]
    for a in poss_a_values:
        for e in range(0, P+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, status = predictor.run()
            if status == 'PASS':
                xpass.append(a)
                ypass.append(e)
            elif status == 'FAIL_SS':
                xssfail.append(a)
                yssfail.append(e)
            elif status == 'FAIL_IC':
                xicfail.append(a)
                yicfail.append(e)
            elif status == 'FAIL_MENTOR':
                xmentorfail.append(a)
                ymentorfail.append(e)
            else:
                xyfr.append(a)
                yyfr.append(e)


    plt.scatter(xpass, ypass, c='green', label='Successes')
    plt.scatter(xssfail, yssfail, c='red', label='SS Failures')
    plt.scatter(xicfail, yicfail, c='firebrick', label='IC Failures')
    plt.scatter(xmentorfail, ymentorfail, c='lightcoral', label='Mentor-Limited Crashes')
    plt.scatter(xyfr, yyfr, c='blue', label='YFR')
    xP = np.array(P)
    plt.axline((0,P-i), slope=0, linestyle='dashed', label='E+I=P', c='black')
    plt.legend(loc='upper right')

    plt.title('Predicted Stability of the System Depending on \nInitial Experienced Pilots and Attrition Rate')
    plt.xlabel('a')
    plt.ylabel('E0')
  
widgets.interact_manual(plot_predictions, N=widgets.IntSlider(value=25, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               I=widgets.IntSlider(value=10, min=0, max=100))


interactive(children=(IntSlider(value=10, description='i', max=30, min=-10), IntSlider(value=25, description='…

<function __main__.plot_predictions(i=10, N=25, P=30, R=250, g=4, h=1, rI=144, rE=144, rY=10000)>

In [9]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

def plot_predictions(e=10, N=25, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,5)

    xpass = []
    ypass = []
    xssfail = []
    yssfail = []
    xicfail = []
    yicfail = []
    xmentorfail = []
    ymentorfail = []
    xyfr = []
    yyfr = []
    poss_a_values = [x/100 for x in range(0,30)]
    for a in poss_a_values:
        for i in range(0, P+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, status = predictor.run()
            if status == 'PASS':
                xpass.append(a)
                ypass.append(i)
            elif status == 'FAIL_SS':
                xssfail.append(a)
                yssfail.append(i)
            elif status == 'FAIL_IC':
                xicfail.append(a)
                yicfail.append(i)
            elif status == 'FAIL_MENTOR':
                xmentorfail.append(a)
                ymentorfail.append(i)
            else:
                xyfr.append(a)
                yyfr.append(i)


    plt.scatter(xpass, ypass, c='green', label='Successes')
    plt.scatter(xssfail, yssfail, c='red', label='SS Failures')
    plt.scatter(xicfail, yicfail, c='firebrick', label='IC Failures')
    plt.scatter(xmentorfail, ymentorfail, c='lightcoral', label='Mentor-Limited Crashes')
    plt.scatter(xyfr, yyfr, c='blue', label='YFR')
    xP = np.array(P)
    plt.axline((0,P-e), slope=0, linestyle='dashed', label='E+I=P', c='black')
    plt.legend(loc='upper right')

    plt.title('Predicted Stability of the System Depending on \nInitial Inexperienced Pilots and Attrition Rate')
    plt.xlabel('a')
    plt.ylabel('I0')
  
widgets.interact_manual(plot_predictions, N=widgets.IntSlider(value=25, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               E=widgets.IntSlider(value=10, min=0, max=100))


interactive(children=(IntSlider(value=10, description='e', max=30, min=-10), IntSlider(value=25, description='…

<function __main__.plot_predictions(e=10, N=25, P=30, R=250, g=4, h=1, rI=144, rE=144, rY=10000)>

In [10]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

def plot_predictions(i=10, a=0.07, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,5)

    xpass = []
    ypass = []
    xssfail = []
    yssfail = []
    xicfail = []
    yicfail = []
    xmentorfail = []
    ymentorfail = []
    xyfr = []
    yyfr = []
    for N in range(0, P+1):
        for e in range(0, P+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, status = predictor.run()
            if status == 'PASS':
                xpass.append(N)
                ypass.append(e)
            elif status == 'FAIL_SS':
                xssfail.append(N)
                yssfail.append(e)
            elif status == 'FAIL_IC':
                xicfail.append(N)
                yicfail.append(e)
            elif status == 'FAIL_MENTOR':
                xmentorfail.append(N)
                ymentorfail.append(e)
            else:
                xyfr.append(N)
                yyfr.append(e)


    plt.scatter(xpass, ypass, c='green', label='Successes')
    plt.scatter(xssfail, yssfail, c='red', label='SS Failures')
    plt.scatter(xicfail, yicfail, c='firebrick', label='IC Failures')
    plt.scatter(xmentorfail, ymentorfail, c='lightcoral', label='Mentor-Limited Crashes')
    plt.scatter(xyfr, yyfr, c='blue', label='YFR')
    xP = np.array(P)
    plt.axline((0,P-i), slope=0, linestyle='dashed', label='E+I=P', c='black')
    plt.legend()

    plt.title('Predicted Stability of the System Depending on \nInitial Experienced and Non-Operational Positions')
    plt.xlabel('N')
    plt.ylabel('E0')
  
widgets.interact_manual(plot_predictions, a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               I=widgets.IntSlider(value=10, min=0, max=10))


interactive(children=(IntSlider(value=10, description='i', max=30, min=-10), FloatSlider(value=0.07, descripti…

<function __main__.plot_predictions(i=10, a=0.07, P=30, R=250, g=4, h=1, rI=144, rE=144, rY=10000)>

In [4]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

def plot_predictions(e=10, a=0.07, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,5)

    xpass = []
    ypass = []
    xssfail = []
    yssfail = []
    xicfail = []
    yicfail = []
    xmentorfail = []
    ymentorfail = []
    xyfr = []
    yyfr = []
    for N in range(0, P+1):
        for i in range(0, P+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, status = predictor.run()
            if status == 'PASS':
                xpass.append(N)
                ypass.append(i)
            elif status == 'FAIL_SS':
                xssfail.append(N)
                yssfail.append(i)
            elif status == 'FAIL_IC':
                xicfail.append(N)
                yicfail.append(i)
            elif status == 'FAIL_MENTOR':
                xmentorfail.append(N)
                ymentorfail.append(i)
            else:
                xyfr.append(N)
                yyfr.append(i)


    plt.scatter(xpass, ypass, c='green', label='Successes')
    plt.scatter(xssfail, yssfail, c='red', label='SS Failures')
    plt.scatter(xicfail, yicfail, c='firebrick', label='IC Failures')
    plt.scatter(xmentorfail, ymentorfail, c='lightcoral', label='Mentor-Limited Crashes')
    plt.scatter(xyfr, yyfr, c='blue', label='YFR')
    xP = np.array(P)
    plt.axline((0,P-e), slope=0, linestyle='dashed', label='E+I=P', c='black')
    plt.legend()

    plt.title('Predicted Stability of the System Depending on \nInitial Experienced and Non-Operational Positions')
    plt.xlabel('N')
    plt.ylabel('I0')
  
widgets.interact_manual(plot_predictions, a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               e=widgets.IntSlider(value=10, min=0, max=100))


interactive(children=(IntSlider(value=10, description='e'), FloatSlider(value=0.07, description='a', max=1.0, …

<function __main__.plot_predictions(e=10, a=0.07, P=30, R=250, g=4, h=1, rI=144, rE=144, rY=10000)>

In [6]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

def plot_predictions(e=10, i=10, P=30, R=250, g=4,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,5)

    xpass = []
    ypass = []
    xssfail = []
    yssfail = []
    xicfail = []
    yicfail = []
    xmentorfail = []
    ymentorfail = []
    xyfr = []
    yyfr = []
    poss_a_values = [x/100 for x in range(0,30)]
    for N in range(0, P+1):
        for a in poss_a_values:
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            _, status = predictor.run()
            if status == 'PASS':
                xpass.append(N)
                ypass.append(a)
            elif status == 'FAIL_SS':
                xssfail.append(N)
                yssfail.append(a)
            elif status == 'FAIL_IC':
                xicfail.append(N)
                yicfail.append(a)
            elif status == 'FAIL_MENTOR':
                xmentorfail.append(N)
                ymentorfail.append(a)
            else:
                xyfr.append(N)
                yyfr.append(a)


    plt.scatter(xpass, ypass, c='green', label='Successes')
    plt.scatter(xssfail, yssfail, c='red', label='SS Failures')
    plt.scatter(xicfail, yicfail, c='firebrick', label='IC Failures')
    plt.scatter(xmentorfail, ymentorfail, c='lightcoral', label='Mentor-Limited Crashes')
    plt.scatter(xyfr, yyfr, c='blue', label='YFR')
    xP = np.array(P)
    #plt.axline((0,P-e), slope=0, linestyle='dashed', label='E+I=P', c='black')
    plt.legend()

    plt.title('Predicted Stability of the System Depending on \nAttrition Rate and Non-Operational Positions')
    plt.xlabel('N')
    plt.ylabel('a')
  
widgets.interact_manual(plot_predictions, i=widgets.IntSlider(value=10, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000),
               e=widgets.IntSlider(value=10, min=0, max=100))


interactive(children=(IntSlider(value=10, description='e'), IntSlider(value=10, description='i'), IntSlider(va…

<function __main__.plot_predictions(e=10, i=10, P=30, R=250, g=4, h=1, rI=144, rE=144, rY=10000)>

In [3]:
%matplotlib inline
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt

def plot_predictions(N=25, P=30, R=250, g=4, a=0.07,
                     h=1, rI=144, rE=144, rY=10000):
    
    plt.rcParams["figure.figsize"] = (8,5)

    times = np.empty([P+1,P+1])
    
    for i in range(0, P+1):
        for e in range(0, P+1):
            predictor = StabilityPredictor(i, e, N, P, R, g/52, a/52, h/52, rI/52, rE/52, rY/52)
            time_in_danger, _ = predictor.run()
            if time_in_danger == np.inf:
                time_in_danger = 500
            times[e,i] = time_in_danger

    plt.imshow(times, cmap='Reds', interpolation='spline16', origin='lower')
    plt.colorbar()
    plt.axline((0,P), slope=-1, linestyle='dashed', label='E+I=P', c='black')

    plt.title('Times in Danger Zones Depending on Initial Experienced and Inexperienced Pilots\n\
    (500 indicates a failed instance)')
    plt.xlabel('I0')
    plt.ylabel('E0')
  
widgets.interact_manual(plot_predictions, N=widgets.IntSlider(value=25, min=0, max=100),
               P=widgets.IntSlider(value=30, min=0, max=100),
               R=widgets.IntSlider(value=250, min=0, max=1000),
               g=widgets.IntSlider(value=4, min=0, max=100),
               a=widgets.FloatSlider(value=0.07, min=0, max=1, step=0.01),
               h=widgets.IntSlider(value=1, min=0, max=100),
               rI=widgets.IntSlider(value=144, min=0, max=1000),
               rE=widgets.IntSlider(value=144, min=0, max=1000),
               rY=widgets.IntSlider(value=10000, min=0, max=100000))



interactive(children=(IntSlider(value=25, description='N'), IntSlider(value=30, description='P'), IntSlider(va…

<function __main__.plot_predictions(N=25, P=30, R=250, g=4, a=0.07, h=1, rI=144, rE=144, rY=10000)>